### LA 311 Data Analysis - 2022

In [ ]:
# Import Plugins (JupyterLite)
import piplite
await piplite.install("pandas")
await piplite.install("numpy")
await piplite.install("pyarrow")
await piplite.install("seaborn")
await piplite.install("folium")

# Import Plugins (Jupyter Notebook)
import pandas as pd
import numpy as np
import pyarrow.parquet as pq
import seaborn as sns
import matplotlib.pyplot as plt
import folium
import folium.plugins as plugins

#### Append monthly parquet data files into full year database

In [ ]:
# List of file paths for each month
file_paths = [
    '2022_1.parquet',
    '2022_2.parquet',
    '2022_3.parquet',
    '2022_4.parquet',
    '2022_5.parquet',
    '2022_6.parquet',
    '2022_7.parquet',
    '2022_8.parquet',
    '2022_9.parquet',
    '2022_10.parquet',
    '2022_11.parquet',
    '2022_12.parquet'
]

# Create an empty DataFrame to store the combined data
df = pd.DataFrame()

# Iterate over the file paths and append the data
for file_path in file_paths:
    # Read the Parquet file for each month
    data = pq.read_table(file_path).to_pandas()
    
    # Convert 'ZipCode' column to string type
    data['ZipCode'] = data['ZipCode'].astype(str)
    
    # Append the data to the combined DataFrame
    df = pd.concat([df, data], ignore_index=True)

# Save the combined data as a new Parquet file representing a full year
df.to_parquet('2022FY.parquet', engine='pyarrow')

#### 311 Data by Request Type - 2022

In [ ]:
# Read Parquet file into DataFrame
df = pd.read_parquet('2022FY.parquet')

# Set the seaborn style
sns.set(style="whitegrid")

# Get the value counts and sort them
RT_counts = df['RequestType'].value_counts().sort_values(ascending=False)

# Plot the bar chart
ax = sns.countplot(x='RequestType', data=df, order=RT_counts.index)

# Rotate x-axis labels
ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha='right')

# Add labels and title
plt.xlabel('Request Type')
plt.ylabel('Count of 311 Requests')
plt.title('311 Data by Request Type 2022')

# Show the plot
plt.show()

#### 311 Data by LA Neighboorhood Council Name - 2022

In [ ]:
# Read Parquet file into DataFrame
df = pd.read_parquet('2022FY.parquet')

# Set the seaborn style
sns.set(style="white")

# Get the value counts and sort them
NC_counts = df['NCName'].value_counts().sort_values(ascending=False)

# Set the figure size
plt.figure(figsize=(24, 8))

# Plot the bar chart
ax = sns.countplot(x='NCName', data=df, order=NC_counts.index, color='steelblue')

# Rotate x-axis labels
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha='right')

# Add labels and title
plt.xlabel('Neighborhood Council Name')
plt.ylabel('Count of 311 Requests')
plt.title('311 Data by LA Neighboorhood Council Name 2022')

# Show the plot
plt.show()

#### 311 Data Heatmapping - 2022

In [ ]:
# Load the data from excel into a pandas dataframe
df = pd.read_parquet('2022FY.parquet')

# Remove NaN values from latitude and longitude
remove_nan = ['Latitude', 'Longitude']
df_LL = df.dropna(subset=remove_nan)

# Create a map centered on the mean latitude and longitude
map_center = [df_LL['Latitude'].mean(), df_LL['Longitude'].mean()]
m = folium.Map(location=map_center, zoom_start=16)

# Create a list of latitude-longitude pairs
locations = df_LL[['Latitude', 'Longitude']].values.tolist()

# Add the heatmap layer to the map
heat_map = folium.plugins.HeatMap(locations)
heat_map.add_to(m)

# Display the map
m

#### 311 Request Types and Sources Cross-referencing

In [ ]:
# Load the data from excel into a pandas dataframe
df = pd.read_parquet('2022FY.parquet')

# Group the DataFrame by both 'Column1' and 'Column2' and calculate the value counts
req_counts = df.groupby(['RequestType', 'RequestSource']).size().reset_index(name='Count')

# Pivot the data
pivot = req_counts.pivot(index='RequestType', columns='RequestSource', values='Count')

# Display the pivoted table
pivot

In [ ]:
# Create a heatmap of the pivoted table
plt.figure(figsize=(14, 6))
sns.heatmap(pivot, annot=True, cmap='coolwarm', fmt='g')

# Set the plot title
plt.title('Heatmap of Request Types and Sources')

# Show the plot
plt.show()